In [ ]:
%load_ext autoreload
%autoreload 2

from utils.data_loading import load_all_color_images, load_all_depth_images, read_trajectory
import rerun as rr
import numpy as np

In [ ]:
import rerun.blueprint as rrb
def setup_blueprint():
    blueprint = rrb.Blueprint(
        rrb.Horizontal(
                rrb.Vertical(
                    rrb.Spatial3DView(name="3D", origin="world", line_grid=rrb.archetypes.LineGrid3D(plane=rr.components.Plane3D(np.array([0, 0, 1])))),
                ),
                rrb.Vertical(
                    rrb.Spatial2DView(name="RGB", origin="world/camera/image", contents="world/camera/image/rgb"),
                    rrb.Spatial2DView(name="Depth", origin="world/camera/image", contents="world/camera/image/depth"),
                    rrb.Spatial2DView(name="SAM", origin="world/camera/image", contents="world/camera/image/sam"),
                    name="2D",
                    row_shares=[1,1,1]
                ),
            ),
    )
    return blueprint

rr.init("something5", spawn=False, default_blueprint=setup_blueprint())
rr.connect_grpc()

In [ ]:
COLOR_PATH = "/local/home/dkorth/Projects/dynamic-scene-graphs/data/living_room_1/color"
DEPTH_PATH = "/local/home/dkorth/Projects/dynamic-scene-graphs/data/living_room_1/depth"
TRAJ_PATH = "/local/home/dkorth/Projects/dynamic-scene-graphs/data/living_room_1/livingroom1-traj.txt"

rgb = load_all_color_images(COLOR_PATH)
depth = load_all_depth_images(DEPTH_PATH)
traj = read_trajectory(TRAJ_PATH)

In [ ]:
from segment import SAM2Segmenter
segmenter = SAM2Segmenter(
    sam2_checkpoint="../checkpoints/sam2.1/sam2.1_hiera_tiny.pt",
    model_cfg="configs/sam2.1/sam2.1_hiera_t.yaml"
)

In [ ]:
import rerun as rr
import seaborn as sns

rr.set_time(timeline="world/camera/image", sequence=0)
K = np.array([[481.20, 0, 319.50], [0, 480.00, 239.50], [0, 0, 1]])

rr.log("world/camera/image", rr.ViewCoordinates.RDF)

rr.log("world/camera/image", rr.Pinhole(
    resolution=[640, 480],
    principal_point=[319.50, 239.50],
    focal_length=[481.20, 480.00],
))

for i, (d, img) in enumerate(zip(depth, rgb[:40])):

    # images
    rr.log("world/camera/image/rgb", rr.Image(img, color_model=rr.ColorModel.RGB))
    rr.log("world/camera/image/depth", rr.DepthImage(d/1000))

    # sam masks
    masks = segmenter.segment(img)
    masks_with_ids = list(enumerate(masks, start=1))
    masks_with_ids.sort(key=(lambda x: x[1]["area"]), reverse=True)

    # Layer all of the masks together, using the id as class-id in the segmentation
    segmentation_img = np.zeros((rgb[0].shape[0], rgb[0].shape[1]))
    for id, m in masks_with_ids:
        segmentation_img[m["segmentation"]] = id
    
    all_points = []
    all_colors = []
    points = unproject_depth_to_world_grid(d/1000, K, traj[i].rotation, traj[i].translation)
    rr.log("points", rr.Points3D(points.reshape(-1, 3), colors=np.zeros((points.shape[0], 3))))
    for color, mask in zip(sns.color_palette("husl", len(masks)), masks):
        points_1 = points[mask["segmentation"]]
        colors_1 = np.array([color] * len(points_1))
        all_points.append(points_1)
        all_colors.append(colors_1)

    rr.log("semantic_points", rr.Points3D(np.concatenate(all_points), colors=np.concatenate(all_colors)))

    rr.log("world/camera/image/sam", rr.SegmentationImage(segmentation_img.astype(np.uint8)))

    # camera pose
    rr.log("world/camera", rr.Transform3D(
        rotation=rr.RotationAxisAngle(axis=traj[i].rotation_axis_angle, angle=np.linalg.norm(traj[i].rotation_axis_angle)),
        translation=traj[i].translation
    ))

    rr.set_time(timeline="world/camera/image", sequence=i)

In [ ]:
import seaborn as sns
sns.color_palette("husl", 10)

In [ ]:
from utils.data_loading import unproject_depth_to_world_grid

K = np.array([[481.20, 0, 319.50], [0, 480.00, 239.50], [0, 0, 1]])
points = unproject_depth_to_world_grid(depth[0]/1000, K, traj[0].rotation, traj[0].translation)


In [ ]:
points

In [ ]:
points[masks[3]["segmentation"]]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(masks[0]["segmentation"])

In [ ]:
# create an colors array similar to the masks array
points_1 = points[masks[3]['segmentation']]
colorss_1 = np.array([[255, 0, 0]] * len(points_1))

In [ ]:
rr.log("points", rr.Points3D(points.ravel()))

In [ ]:
from utils.data_loading import unproject_depth_to_world

K = np.array([[481.20, 0, 319.50], [0, 480.00, 239.50], [0, 0, 1]])
points = unproject_depth_to_world(depth[0]/1000, K, traj[0].rotation, traj[0].translation)